In [5]:
# 本代码由可视化策略环境自动生成 2021年4月6日11:46
# 本代码单元只能在可视化模式下编辑。您也可以拷贝代码，粘贴到新建的代码单元或者策略，然后修改。


# 回测引擎：初始化函数，只执行一次
def m5_initialize_bigquant_run(context):
    '''
    策略思想：在每月月初等权重买入符合条件的股票，下月月初换仓。
    选股条件：
    pe_ttm>0且最小的80只股票
    流通市值在50亿以下 
    非ST股票
    '''
    
    # 加载股票指标数据,数据继承自m6模块
    context.indicator_data = context.options['data'].read_df()

    # 系统已经设置了默认的交易手续费和滑点，要修改手续费可使用如下函数
    context.set_commission(PerOrder(buy_cost=0.0003, sell_cost=0.0013, min_cost=5))
    
    # 设置股票数量
    context.stock_num = 20
    
    # 调仓天数,22个交易日大概就是一个月。可以理解为一个月换仓一次
    context.rebalance_days = 22
    
    # 如果策略运行中，需要将数据进行保存，可以借用extension这个对象，类型为dict
    # 比如当前运行的k线的索引，比如个股持仓天数、买入均价
    if 'index' not in context.extension:
        context.extension['index'] = 0
 
    

# 回测引擎：每日数据处理函数，每天执行一次
def m5_handle_data_bigquant_run(context, data):
    
    
    context.extension['index'] += 1
    # 不在换仓日就return,相当于后面的代码只会一个月运行一次，买入的股票会持有1个月
    if  context.extension['index'] % context.rebalance_days != 0:
        return 
    
    # 当前的日期
    date = data.current_dt.strftime('%Y-%m-%d')
    
    cur_data = context.indicator_data[context.indicator_data['date'] == date]
    # 根据日期获取调仓需要买入的股票的列表
    stock_to_buy = list(cur_data.instrument[:context.stock_num])
    # 通过positions对象，使用列表生成式的方法获取目前持仓的股票列表
    stock_hold_now = [equity.symbol for equity in context.portfolio.positions]
    # 继续持有的股票：调仓时，如果买入的股票已经存在于目前的持仓里，那么应继续持有
    no_need_to_sell = [i for i in stock_hold_now if i in stock_to_buy]
    # 需要卖出的股票
    stock_to_sell = [i for i in stock_hold_now if i not in no_need_to_sell]
  
    # 卖出
    for stock in stock_to_sell:
        # 如果该股票停牌，则没法成交。因此需要用can_trade方法检查下该股票的状态
        # 如果返回真值，则可以正常下单，否则会出错
        # 因为stock是字符串格式，我们用symbol方法将其转化成平台可以接受的形式：Equity格式

        if data.can_trade(context.symbol(stock)):
            # order_target_percent是平台的一个下单接口，表明下单使得该股票的权重为0，
            #   即卖出全部股票，可参考回测文档
            context.order_target_percent(context.symbol(stock), 0)
    
    # 如果当天没有买入的股票，就返回
    if len(stock_to_buy) == 0:
        return

    # 等权重买入 
    weight =  1 / len(stock_to_buy)
    
    # 买入
    for stock in stock_to_buy:
        if data.can_trade(context.symbol(stock)):
            # 下单使得某只股票的持仓权重达到weight，因为
            # weight大于0,因此是等权重买入
            context.order_target_percent(context.symbol(stock), weight)
 
# 回测引擎：准备数据，只执行一次
def m5_prepare_bigquant_run(context):
    pass

# 回测引擎：每个单位时间开始前调用一次，即每日开盘前调用一次。
def m5_before_trading_start_bigquant_run(context, data):
    pass


m1 = M.instruments.v2(
    start_date='2010-01-01',
    end_date='2021-04-06',
    market='CN_STOCK_A',
    instrument_list='',
    max_count=0
)

m2 = M.input_features.v1(
    features="""market_cap_float_0
amount_0
pe_ttm_0
st_status_0"""
)

m3 = M.general_feature_extractor.v7(
    instruments=m1.data,
    features=m2.data,
    start_date='',
    end_date='',
    before_start_days=60
)

m4 = M.sort.v4(
    input_ds=m3.data,
    sort_by='pe_ttm_0',
    group_by='instrument',
    keep_columns='--',
    ascending=True
)

m6 = M.filter.v3(
    input_data=m4.sorted_data,
    expr='pe_ttm_0>0 & market_cap_float_0 < 5000000000 & st_status_0 == 0',
    output_left_data=False
)

m5 = M.trade.v4(
    instruments=m1.data,
    options_data=m6.data,
    start_date='',
    end_date='',
    initialize=m5_initialize_bigquant_run,
    handle_data=m5_handle_data_bigquant_run,
    prepare=m5_prepare_bigquant_run,
    before_trading_start=m5_before_trading_start_bigquant_run,
    volume_limit=0.025,
    order_price_field_buy='open',
    order_price_field_sell='open',
    capital_base=1000000,
    auto_cancel_non_tradable_orders=True,
    data_frequency='daily',
    price_type='后复权',
    product_type='股票',
    plot_charts=True,
    backtest_only=False,
    benchmark=''
)


[2021-04-06 11:39:38.738368] INFO: moduleinvoker: instruments.v2 开始运行..

[2021-04-06 11:39:38.820258] INFO: moduleinvoker: instruments.v2 运行完成[0.081885s].

[2021-04-06 11:39:38.824303] INFO: moduleinvoker: input_features.v1 开始运行..

[2021-04-06 11:39:38.832398] INFO: moduleinvoker: 命中缓存

[2021-04-06 11:39:38.834086] INFO: moduleinvoker: input_features.v1 运行完成[0.009795s].

...

[2021-04-06 11:41:13.690718] INFO: Performance: Simulated 2735 trading days out of 2735.

[2021-04-06 11:41:13.692258] INFO: Performance: first open: 2010-01-04 09:30:00+00:00

[2021-04-06 11:41:13.693252] INFO: Performance: last close: 2021-04-06 15:00:00+00:00

[2021-04-06 11:41:40.702839] INFO: moduleinvoker: backtest.v8 运行完成[72.408869s].

[2021-04-06 11:41:40.704480] INFO: moduleinvoker: trade.v4 运行完成[74.197859s].